In [1]:
!pip install transformers

     |████████████████████████████████| 368kB 2.7MB/s 
     |████████████████████████████████| 645kB 48.6MB/s 
     |████████████████████████████████| 1.0MB 42kB/s 
     |████████████████████████████████| 860kB 40.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.35-cp36-none-any.whl size=883999 sha256=da5ee58229d35b15605638e54a603a0cb8c6d5f73fd4a1b706af8a93eadadf2e
  Stored in directory: /root/.cache/pip/wheels/63/2a/db/63e2909042c634ef551d0d9ac825b2b0b32dede4a6d87ddc94
Successfully built sacremoses


In [0]:
import torch

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
from torch.utils.data import Dataset, DataLoader
import pandas as pd

class SentimentDataset(Dataset):
	def __init__(self, path_to_file):
		self.dataset = pd.read_csv(path_to_file, sep="\t", names=["text", "label"])
	def __len__(self):
		return len(self.dataset)
	def __getitem__(self, idx):
		text = self.dataset.loc[idx, "text"]
		label = self.dataset.loc[idx, "label"]
		sample = {"text": text, "label": label}
		return sample

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from transformers import BertTokenizer
from transformers import BertForSequenceClassification, BertModel
from torch.utils.data import DataLoader
from transformers import AdamW

In [0]:
# 超参数
hidden_dropout_prob = 0.5
num_labels = 2
learning_rate = 1e-5
weight_decay = 1e-2
epochs = 2
max_len = 100
batch_size = 16
class_num = 2

base_path = "/content/drive/My Drive/Colab Notebooks/"
vocab_file = base_path + "PyTorch_Pretrained_Model/chinese_wwm_pytorch/vocab.txt"

In [8]:
torch.cuda.get_device_name(0)

'Tesla K80'

In [0]:
# 使用GPU
# 然后通过model.to(device)的方式使用
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [0]:
data_path = base_path + "/data/sentiment/"
# 加载数据集
sentiment_train_set = SentimentDataset(data_path + "sentiment.train.data")
sentiment_train_loader = DataLoader(sentiment_train_set, batch_size=batch_size, shuffle=True, num_workers=2)

sentiment_valid_set = SentimentDataset(data_path + "sentiment.train.data")
sentiment_valid_loader = DataLoader(sentiment_valid_set, batch_size=batch_size, shuffle=False, num_workers=2)

In [0]:
# 加载模型
# transformers也可以从网络下载预训练模型
# 如果是用本地路径确认3个文件的名称为config.json，pytorch_model.bin和vocab.txt
model = BertForSequenceClassification.from_pretrained("bert-base-uncased")

In [0]:
# 定义优化器和损失函数
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]
#optimizer = AdamW(model.parameters(), lr=learning_rate)
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
criterion = nn.CrossEntropyLoss()
tokenizer = BertTokenizer(vocab_file)

In [0]:
def convert_text_to_ids(tokenizer, text, max_len=100):
	if isinstance(text, str):
		tokenized_text = tokenizer.encode_plus(text, max_length=max_len, add_special_tokens=True)
		input_ids = tokenized_text["input_ids"]
		token_type_ids = tokenized_text["token_type_ids"]
	elif isinstance(text, list):
		input_ids = []
		token_type_ids = []
		for t in text:
			tokenized_text = tokenizer.encode_plus(t, max_length=max_len, add_special_tokens=True)
			input_ids.append(tokenized_text["input_ids"])
			token_type_ids.append(tokenized_text["token_type_ids"])
	else:
		print("Unexpected input")
	return input_ids, token_type_ids


def seq_padding(tokenizer, X):
	pad_id = tokenizer.convert_tokens_to_ids("[PAD]")
	if len(X) <= 1:
		return X
	L = [len(x) for x in X]
	ML = max(L)
	X = torch.Tensor([x + [pad_id] * (ML - len(x)) if len(x) < ML else x for x in X])
	#X.requires_grad = True
	return X

In [0]:
def train(model, iterator, optimizer, criterion, device, clip):
	model.to(device)
	model.train()
	epoch_loss = 0
	epoch_acc = 0
	for i, batch in enumerate(iterator):
		label = batch["label"]
		text = batch["text"]
		input_ids, token_type_ids = convert_text_to_ids(tokenizer, text)
		input_ids = seq_padding(tokenizer, input_ids)
		token_type_ids = seq_padding(tokenizer, token_type_ids)
		# 标签形状为 (batch_size, 1) 
		label = label.unsqueeze(1)
		# 需要 LongTensor
		input_ids, token_type_ids, label = input_ids.long(), token_type_ids.long(), label.long()
		# 梯度清零
		optimizer.zero_grad()
		# 迁移到GPU
		input_ids, token_type_ids, label = input_ids.to(device), token_type_ids.to(device), label.to(device)
		# (loss), logits, (hidden_states), (attentions)
		# (hidden_states), (attentions) 不一定存在
		output = model(input_ids=input_ids, token_type_ids=token_type_ids, labels=label)
		y_pred_prob = output[1]
		y_pred_label = y_pred_prob.argmax(dim=1)
		# 计算loss
		#loss = criterion(y_pred_prob.view(-1, 2), label.view(-1))
		loss = output[0]
		# 计算acc
		acc = ((y_pred_label == label.view(-1)).sum()).item()
		# 梯度裁剪
		#torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
		# 反向传播
		loss.backward()
		optimizer.step()
		# epoch 中的 loss 和 acc 累加
		epoch_loss += loss.item()
		epoch_acc += acc
		if i % 100 == 0:
			print("current loss:", epoch_loss / (i+1), "\t", "current acc:", epoch_acc / ((i+1)*len(label)))
	return epoch_loss / len(iterator), epoch_acc / len(iterator.dataset.dataset)

def evaluate(model, iterator, criterion, device):
	model.to(device)
	model.eval()
	epoch_loss = 0
	epoch_acc = 0
	with torch.no_grad():
		for _, batch in enumerate(iterator):
			label = batch["label"]
			text = batch["text"]
			input_ids, token_type_ids = convert_text_to_ids(tokenizer, text)
			input_ids = seq_padding(tokenizer, input_ids)
			token_type_ids = seq_padding(tokenizer, token_type_ids)
			# 标签形状为 (batch_size, 1) 
			label = label.unsqueeze(1)
			# 需要 LongTensor
			input_ids, token_type_ids, label = input_ids.long(), token_type_ids.long(), label.long()
			# 迁移到GPU
			input_ids, token_type_ids, label = input_ids.to(device), token_type_ids.to(device), label.to(device)
			output = model(input_ids=input_ids, token_type_ids=token_type_ids, labels=label)
			y_pred_label = output[1].argmax(dim=1)
			loss = output[0]
			acc = ((y_pred_label == label.view(-1)).sum()).item()
			epoch_loss += loss.item()
			epoch_acc += acc
	return epoch_loss / len(iterator), epoch_acc / len(iterator.dataset.dataset)

In [0]:
for i in range(epochs):
	train_loss, train_acc = train(model, sentiment_train_loader, optimizer, criterion, device, 1)
	print("train loss: ", train_loss, "\t", "train acc:", train_acc)
	valid_loss, valid_acc = evaluate(model, sentiment_valid_loader, criterion, device)
	print("valid loss: ", valid_loss, "\t", "valid acc:", valid_acc)

training on GPU
current loss: 0.6535788178443909 	 current acc: 0.6875
current loss: 0.6699029826650432 	 current acc: 0.5857843137254902
current loss: 0.6760076998484017 	 current acc: 0.5792079207920792
current loss: 0.6762414921987925 	 current acc: 0.5823675496688742
current loss: 0.669018224697208 	 current acc: 0.5948383084577115
current loss: 0.6682184906119845 	 current acc: 0.5971115537848606
current loss: 0.6643820257479962 	 current acc: 0.604235880398671
current loss: 0.6600956170477419 	 current acc: 0.6107549857549858
current loss: 0.6605680557409129 	 current acc: 0.6126870324189526
current loss: 0.6558593532066388 	 current acc: 0.614329268292683
current loss: 0.6493636944336806 	 current acc: 0.6227544910179641
current loss: 0.6429205579571629 	 current acc: 0.6306715063520871
current loss: 0.6372712821809702 	 current acc: 0.6363352745424293
current loss: 0.6260670349803023 	 current acc: 0.6469854070660522
current loss: 0.6159548913148603 	 current acc: 0.65575962910

In [0]:
from transformers import BertConfig

In [0]:
config = BertConfig.from_pretrained("bert-base-uncased", num_labels=num_labels, config = BertConfig.from_pretrained("bert-base-uncased", num_labels=num_labels, hidden_dropout_prob=hidden_dropout_prob))
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", config=config)

In [18]:
for i in range(epochs):
	train_loss, train_acc = train(model, sentiment_train_loader, optimizer, criterion, device, 1)
	valid_loss, valid_acc = evaluate(model, sentiment_valid_loader, criterion, device)
	print("\n")
	print("train loss: ", train_loss, "\t", "train acc:", train_acc)
	print("valid loss: ", valid_loss, "\t", "valid acc:", valid_acc, end="\n\n")

current loss: 0.6253945231437683 	 current acc: 0.75
current loss: 0.6724979877471924 	 current acc: 0.594059405940594
current loss: 0.6675496136964257 	 current acc: 0.595771144278607
current loss: 0.6500755327682558 	 current acc: 0.6218853820598007
current loss: 0.6304198570753868 	 current acc: 0.6447942643391521
current loss: 0.6088782434989354 	 current acc: 0.6645459081836327
current loss: 0.5883931425615476 	 current acc: 0.6828202995008319
current loss: 0.5731407172244558 	 current acc: 0.6957917261055635
current loss: 0.5600425008865778 	 current acc: 0.7063046192259675
current loss: 0.5460098291210805 	 current acc: 0.7180216426193119
current loss: 0.5335281072975276 	 current acc: 0.728021978021978


train loss:  0.5258307942637691 	 train acc: 0.7331433998100665
valid loss:  0.32960593094739704 	 valid acc: 0.8585588793922128

current loss: 0.6428508758544922 	 current acc: 0.625
current loss: 0.36294595543110725 	 current acc: 0.8409653465346535
current loss: 0.3547838136

In [0]:
# 保存
model.save_pretrained("./")
tokenizer.save_pretrained("./")